<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

C:\Users\Michael\Anaconda3\envs\U4-S1-NLP\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# Tokenizer function

def tokenize(text):
    tokens = []
    for token in simple_preprocess(text):
        if token not in STOPWORDS and len(token) > 3:
            tokens.append(token)
    return tokens

In [3]:
df = pd.read_csv('data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')

In [5]:
df.head(1)

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."


In [8]:
df['tokens'] = df['reviews.text'].apply(tokenize)

In [9]:
df['tokens'].head()

0    [order, item, quality, missing, backup, spring...
1                    [bulk, expensive, products, like]
2                             [duracell, price, happy]
3              [work, brand, batteries, better, price]
4             [batteries, long, lasting, price, great]
Name: tokens, dtype: object

In [11]:
df = df[['name', 'tokens']]

In [12]:
df.head()

,name,tokens
0,AmazonBasics AAA Performance Alkaline Batterie...,"[order, item, quality, missing, backup, spring..."
1,AmazonBasics AAA Performance Alkaline Batterie...,"[bulk, expensive, products, like]"
2,AmazonBasics AAA Performance Alkaline Batterie...,"[duracell, price, happy]"
3,AmazonBasics AAA Performance Alkaline Batterie...,"[work, brand, batteries, better, price]"
4,AmazonBasics AAA Performance Alkaline Batterie...,"[batteries, long, lasting, price, great]"


In [ ]:
df.name.value_counts()

# k. not sure the name is meaningful... might just chop it.

In [24]:
# Here I just called the df instead of that crazy path method from lecture dealing with 
# a bunch of text files (looks more useful that way, but this is easier...)

id2word = corpora.Dictionary(df['tokens'])

In [22]:
# Checking to see it it works. It do. 

id2word.token2id['battery']

2

In [25]:
# How many words are we talkin'?
len(id2word.keys())

8980

In [26]:
# This is a way of tuning/trimming the words we use

id2word.filter_extremes(no_below=5, no_above=0.9)

In [27]:
# Look at the number of words left after trimming
len(id2word.keys())

3337

In [28]:
# Need a "Bag of Words"

corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [30]:
corpus[6][:10]

[(15, 1), (36, 1), (37, 1), (38, 1)]

In [31]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=5,
                   num_topics = 15,
                   passes=10,
                   workers=4
                  )

In [32]:
# Oooooo "topics"
lda.print_topics()

[(0,
  '0.077*"great" + 0.053*"deal" + 0.040*"batteries" + 0.034*"best" + 0.028*"item" + 0.018*"love" + 0.018*"easy" + 0.015*"come" + 0.014*"convenient" + 0.014*"price"'),
 (1,
  '0.149*"batteries" + 0.056*"long" + 0.038*"work" + 0.036*"good" + 0.034*"brand" + 0.031*"price" + 0.025*"brands" + 0.022*"great" + 0.020*"amazon" + 0.018*"time"'),
 (2,
  '0.073*"kids" + 0.042*"tablet" + 0.036*"love" + 0.033*"year" + 0.029*"apps" + 0.023*"games" + 0.021*"time" + 0.020*"great" + 0.019*"easy" + 0.018*"child"'),
 (3,
  '0.057*"batteries" + 0.024*"battery" + 0.019*"amazon" + 0.010*"time" + 0.010*"charge" + 0.009*"bought" + 0.009*"remote" + 0.009*"long" + 0.009*"months" + 0.009*"work"'),
 (4,
  '0.109*"kindle" + 0.024*"light" + 0.019*"reading" + 0.018*"bought" + 0.016*"love" + 0.016*"reader" + 0.015*"read" + 0.013*"screen" + 0.012*"ipad" + 0.012*"easy"'),
 (5,
  '0.050*"happy" + 0.035*"purchase" + 0.022*"product" + 0.016*"thank" + 0.016*"satisfied" + 0.015*"service" + 0.014*"couldn" + 0.014*"purcha

In [33]:
# RegEx away all the extra silliness from the above output

words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [34]:
topics = [' '.join(t[0:5]) for t in words]

In [35]:
# Pretty print-job of topics without all the silly extra stuff

for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
great deal batteries best item

------ Topic 1 ------
batteries long work good brand

------ Topic 2 ------
kids tablet love year apps

------ Topic 3 ------
batteries battery amazon time charge

------ Topic 4 ------
kindle light reading bought love

------ Topic 5 ------
happy purchase product thank satisfied

------ Topic 6 ------
easy tablet size perfect expected

------ Topic 7 ------
tablet loves great bought year

------ Topic 8 ------
amazon good quality amazing pretty

------ Topic 9 ------
books games read bought reading

------ Topic 10 ------
great good price product works

------ Topic 11 ------
tablet amazon great nice prime

------ Topic 12 ------
tablet apps amazon store google

------ Topic 13 ------
screen user clear friendly better

------ Topic 14 ------
love great tablet size price



In [36]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

C:\Users\Michael\Anaconda3\envs\U4-S1-NLP\lib\site-packages\past\types\oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable


In [37]:
# This is going to become our visualizer 

pyLDAvis.gensim.prepare(lda, corpus, id2word)

C:\Users\Michael\Anaconda3\envs\U4-S1-NLP\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.287135 -0.036175       1        1  12.030126
3      0.184927  0.145763       2        1  10.937418
2     -0.189726 -0.094655       3        1   9.100093
12    -0.047959  0.026887       4        1   7.761092
10     0.190818 -0.257894       5        1   7.377483
4     -0.098951  0.151624       6        1   7.232648
7     -0.202768 -0.189058       7        1   6.975447
11    -0.028959 -0.075558       8        1   6.565312
9     -0.203903  0.025494       9        1   6.251534
14    -0.065715 -0.007245      10        1   5.799354
6     -0.116295  0.038506      11        1   5.007586
8      0.115303  0.044497      12        1   4.229271
0      0.130067 -0.083622      13        1   4.006935
5      0.069554  0.136734      14        1   3.433861
13    -0.023528  0.174700      15        1   3.291851, topic_info=     Category         Freq       Term        Total  loglift  logprob
15    Default  7195.000000  batteries  7195.000000  30.0000  30.0000
18    Default  9020.000000      great  9020.000000  29.0000  29.0000
30    Default  5561.000000       good  5561.000000  28.0000  28.0000
14    Default  4795.000000      price  4795.000000  27.0000  27.0000
2255  Default  6952.000000     tablet  6952.000000  26.0000  26.0000
1928  Default  3125.000000     kindle  3125.000000  25.0000  25.0000
1721  Default  2126.000000      loves  2126.000000  24.0000  24.0000
694   Default  3076.000000       easy  3076.000000  23.0000  23.0000
170   Default  2428.000000       kids  2428.000000  22.0000  22.0000
79    Default  3493.000000       love  3493.000000  21.0000  21.0000
1557  Default  1737.000000     screen  1737.000000  20.0000  20.0000
1939  Default  1440.000000      books  1440.000000  19.0000  19.0000
147   Default  1997.000000    product  1997.000000  18.0000  18.0000
20    Default  2463.000000       long  2463.000000  17.0000  17.0000
90    Default  1769.000000       year  1769.000000  16.0000  16.0000
1435  Default  1808.000000       apps  1808.000000  15.0000  15.0000
17    Default  1245.000000      brand  1245.000000  14.0000  14.0000
599   Default  1727.000000      games  1727.000000  13.0000  13.0000
390   Default   911.000000       deal   911.000000  12.0000  12.0000
818   Default  1018.000000       gift  1018.000000  11.0000  11.0000
724   Default  1205.000000       read  1205.000000  10.0000  10.0000
25    Default  3131.000000     bought  3131.000000   9.0000   9.0000
876   Default  1180.000000    reading  1180.000000   8.0000   8.0000
13    Default   847.000000      happy   847.000000   7.0000   7.0000
21    Default  3840.000000     amazon  3840.000000   6.0000   6.0000
84    Default  1461.000000      works  1461.000000   5.0000   5.0000
346   Default   947.000000      value   947.000000   4.0000   4.0000
63    Default   853.000000     brands   853.000000   3.0000   3.0000
27    Default   854.000000  christmas   854.000000   2.0000   2.0000
1173  Default   923.000000   daughter   923.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
1557  Topic15   688.517395     screen  1737.097656   2.4883  -2.5760
3041  Topic15    29.959482   magazine    39.966316   3.1255  -5.7107
2917  Topic15    50.435463  processor    80.977646   2.9402  -5.1899
584   Topic15   136.584457   problems   297.734436   2.6345  -4.1936
1553  Topic15    49.874065      sharp    80.422180   2.9359  -5.2011
2851  Topic15    23.333534      swipe    31.158283   3.1245  -5.9607
340   Topic15    35.980873    trouble    59.734184   2.9068  -5.5276
2794  Topic15    38.177349  magazines    67.911942   2.8378  -5.4683
332   Topic15    84.969254      super   253.878708   2.3192  -4.6683
16    Topic15   228.137268     better  1452.055420   1.5629  -3.6806
245   Topic15    53.014805     bright   124.740158   2.5581  -5.1400
2641  Topic15    37.486423      crisp    66.843704   2.8353  -5.4866
191

### Quick Recap / reflections

* The topics here seem to be ligit: not overlapping and approximately the same size. 
* All of the summarized topics are overwhelmingly positive
* Were these real people or Amazon bots? It seems almost too perfect... When I go to buy a product, the distribution of reviews makes a big difference. Amazon products get such great reviews. I guess that it most likely has much more to do with their smart business practices -- they don't need bots, they predict which products will be liked based upon this sort of anaylsis. 